# Merge All Layers with Public Land Fields

In [ ]:
import arcpy
import os
ws = r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\TaxLots\TaxLots.gdb'
arcpy.env.workspace = ws
arcpy.env.overwriteOutput = True

In [ ]:
outfc = r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\TaxLots\TaxLots.gdb\allCounties'

clat = 'clatsopParcels'
col = 'colParcels'
cow = 'cowlitzParcels'
mult = 'multnomahParcels'
pac = 'pacificParcels'
ska = 'skaParcels'
wah = 'wahkiakumParcels'
clark = 'clarkParcels'

coList = [clat, col, cow, mult, pac, ska, wah, clark]

clatOwn = 'OWNER_LINE'
colOwn = 'OWNER'
cowOwn = 'DEED_HOLDE'
multOwn = 'OWNER1'
pacOwn = 'CURRENT_O'
skaOwn = 'OwnerLast'
wahOwn = 'Property_O'
clarkOwn = 'PACSOwner'

ownList = [clatOwn, colOwn, cowOwn, multOwn, pacOwn, skaOwn, wahOwn, clarkOwn]

publicLand = 'PublicLand'
AREA = 'Shape_Area'
LENGTH = 'Shape_Length'




In [ ]:
publicLandFldMap = arcpy.FieldMap()
shapeAreaFldMap = arcpy.FieldMap()
shapeLengthFldMap = arcpy.FieldMap()
landOwnerFldMap = arcpy.FieldMap()

fms = arcpy.FieldMappings()

In [ ]:
for co, own in zip(coList, ownList):
    publicLandFldMap.addInputField(co, publicLand)
    shapeAreaFldMap.addInputField(co, AREA)
    shapeLengthFldMap.addInputField(co, LENGTH)
    landOwnerFldMap.addInputField(co, own)
    

In [ ]:
landOwner_name = landOwnerFldMap.outputField
landOwner_name.name = 'Land Owner'
landOwner_name.length = 100
landOwnerFldMap.outputField = landOwner_name

In [ ]:
area_name = shapeAreaFldMap.outputField
area_name.name = 'AREA'
shapeAreaFldMap.outputField = area_name

length_name = shapeLengthFldMap.outputField
length_name.name = 'LENGTH'
shapeLengthFldMap.outputField = length_name

In [ ]:
fms.addFieldMap(publicLandFldMap)
fms.addFieldMap(shapeAreaFldMap)
fms.addFieldMap(shapeLengthFldMap)
fms.addFieldMap(landOwnerFldMap)

In [ ]:
arcpy.Merge_management(coList, 'county_merge', fms)

# Filter out public land owners

In [ ]:
arcpy.Select_analysis('county_merge', 'allCoPublicLands', "PublicLand = 'T'")

# Create Flood Plain Intersect Layer

In [ ]:
floodPlain = r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\FloodPlain\Hydrogeo_III_merged.shp'

In [ ]:
pubFloodPlain = arcpy.SelectLayerByLocation_management('allCoPublicLands', overlap_type = 'INTERSECT', select_features = floodPlain)
arcpy.CopyFeatures_management(pubFloodPlain, 'publicLandsFloodplain')

# Next script is classifyPublicLands